In [0]:
# Cell 1: Install required libraries
%pip install openai python-dotenv PyPDF2
dbutils.library.restartPython()

In [0]:
# Cell 2: Azure OpenAI Configuration
import os

# Canada East - Embedding Model
AZURE_OPENAI_KEY = ""
AZURE_OPENAI_ENDPOINT = ""
AZURE_OPENAI_API_VERSION = ""
EMBEDDING_DEPLOYMENT = ""

# East US 2 - GPT Model
AZURE_OPENAI_KEY_EASTUS2 = ""
AZURE_OPENAI_ENDPOINT_EASTUS2 = ""
GPT_DEPLOYMENT = ""

print("Configuration loaded")

In [0]:
# Cell 3: Test Azure OpenAI from Databricks
from openai import AzureOpenAI

# Initialize clients
client_canada = AzureOpenAI(
    api_key=AZURE_OPENAI_KEY,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT
)

client_eastus = AzureOpenAI(
    api_key=AZURE_OPENAI_KEY_EASTUS2,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT_EASTUS2
)

# Test embedding
test_embedding = client_canada.embeddings.create(
    model=EMBEDDING_DEPLOYMENT,
    input="Test from Databricks"
)

print(f"Embedding works! Dimension: {len(test_embedding.data[0].embedding)}")

# Test GPT
test_chat = client_eastus.chat.completions.create(
    model=GPT_DEPLOYMENT,
    messages=[{"role": "user", "content": "Say hello from Databricks!"}],
    max_tokens=50
)

print(f"GPT works! Response: {test_chat.choices[0].message.content}")

In [0]:
# Cell 4: Create database structure
spark.sql("CREATE DATABASE IF NOT EXISTS energy_rag")
spark.sql("USE energy_rag")

print("Database 'energy_rag' created and active")

In [0]:
# Cell 5: Download sample energy documents
import requests
import os

# Create directory for PDFs - use /tmp for temporary storage
pdf_dir = "/tmp/energy_docs"
os.makedirs(pdf_dir, exist_ok=True)

# Sample energy documents (public sources)
documents = {
    "aer_directive_017.pdf": "https://static.aer.ca/prd/documents/directives/Directive017.pdf",
    "aer_directive_060.pdf": "https://static.aer.ca/prd/documents/directives/Directive060.pdf"
}

for filename, url in documents.items():
    filepath = os.path.join(pdf_dir, filename)
    if not os.path.exists(filepath):
        print(f"Downloading {filename}...")
        response = requests.get(url, timeout=30)
        with open(filepath, 'wb') as f:
            f.write(response.content)
        print(f"  Saved to {filepath}")
    else:
        print(f"  {filename} already exists")

print(f"\nTotal documents: {len(os.listdir(pdf_dir))}")

In [0]:
# Cell 6: Extract text from PDFs
from PyPDF2 import PdfReader
import pandas as pd

def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text

# Extract text from all PDFs
documents_data = []

for filename in os.listdir(pdf_dir):
    if filename.endswith('.pdf'):
        filepath = os.path.join(pdf_dir, filename)
        print(f"Processing {filename}...")
        
        text = extract_text_from_pdf(filepath)
        
        documents_data.append({
            'document_id': filename.replace('.pdf', ''),
            'document_name': filename,
            'full_text': text,
            'char_count': len(text)
        })
        
        print(f"  Extracted {len(text)} characters")

# Create DataFrame
df_docs = pd.DataFrame(documents_data)
print(f"\nProcessed {len(df_docs)} documents")
print(df_docs[['document_name', 'char_count']])

In [0]:
# Cell 7: Chunk documents into smaller pieces
def chunk_text(text, chunk_size=1000, overlap=200):
    chunks = []
    start = 0
    
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        start = end - overlap
    
    return chunks

# Create chunks for all documents
all_chunks = []

for idx, row in df_docs.iterrows():
    doc_id = row['document_id']
    doc_name = row['document_name']
    full_text = row['full_text']
    
    chunks = chunk_text(full_text, chunk_size=1000, overlap=200)
    
    for chunk_idx, chunk_content in enumerate(chunks):
        all_chunks.append({
            'document_id': doc_id,
            'document_name': doc_name,
            'chunk_id': f"{doc_id}_chunk_{chunk_idx}",
            'chunk_index': chunk_idx,
            'text': chunk_content,
            'char_count': len(chunk_content)
        })
    
    print(f"{doc_name}: {len(chunks)} chunks created")

df_chunks = pd.DataFrame(all_chunks)
print(f"\nTotal chunks: {len(df_chunks)}")
print(f"Sample chunk:\n{df_chunks.iloc[0]['text'][:200]}...")

In [0]:
# Cell 8: Generate embeddings for all chunks
import time
from tqdm import tqdm

def get_embedding(text):
    response = client_canada.embeddings.create(
        model=EMBEDDING_DEPLOYMENT,
        input=text
    )
    return response.data[0].embedding

# Generate embeddings in batches
embeddings = []
batch_size = 50

print(f"Generating embeddings for {len(df_chunks)} chunks...")
print("This will take about 3-5 minutes and cost approximately $2-3")

for i in tqdm(range(0, len(df_chunks), batch_size)):
    batch = df_chunks.iloc[i:i+batch_size]
    
    for idx, row in batch.iterrows():
        embedding = get_embedding(row['text'])
        embeddings.append(embedding)
    
    # Small delay to avoid rate limits
    time.sleep(0.5)

df_chunks['embedding'] = embeddings
print(f"\nCompleted! Generated {len(embeddings)} embeddings")
print(f"Embedding dimension: {len(embeddings[0])}")

In [0]:
# Cell 9: Convert to Spark DataFrame and save to Delta
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, FloatType

# Convert pandas DataFrame to Spark DataFrame
spark_df = spark.createDataFrame(df_chunks)

# Save as Delta table in Unity Catalog
table_name = "energy_rag.document_chunks"

spark_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable(table_name)

print(f"Saved {spark_df.count()} chunks to {table_name}")

# Verify the table
result = spark.sql(f"SELECT COUNT(*) as total FROM {table_name}").collect()
print(f"Verified: {result[0]['total']} rows in table")

In [0]:
%sql
SELECT * FROM energy_rag.document_chunks;

In [0]:
# Cell 10a: Install Vector Search library
%pip install databricks-vectorsearch
dbutils.library.restartPython()

In [0]:
# Cell 10: Create Vector Search Index
from databricks.vector_search.client import VectorSearchClient

# Initialize Vector Search client
vsc = VectorSearchClient()

# Create vector search endpoint (if not exists)
endpoint_name = "energy_rag_endpoint"

try:
    vsc.create_endpoint(name=endpoint_name)
    print(f"Created endpoint: {endpoint_name}")
except Exception as e:
    print(f"Endpoint already exists or error: {e}")

# Wait for endpoint to be ready
import time
time.sleep(30)

print("Endpoint ready")

In [0]:
# Cell 11b: Check full table name with catalog
result = spark.sql("DESCRIBE EXTENDED energy_rag.document_chunks").collect()

# Find the catalog info
for row in result:
    if 'Catalog' in str(row):
        print(row)

# Also check current catalog
current_catalog = spark.sql("SELECT current_catalog()").collect()[0][0]
print(f"Current catalog: {current_catalog}")

In [0]:
# Cell 11c: Enable Change Data Feed on the table
spark.sql("""
    ALTER TABLE databricks_energy_rag.energy_rag.document_chunks 
    SET TBLPROPERTIES (delta.enableChangeDataFeed = true)
""")

print("Change Data Feed enabled on document_chunks table")

In [0]:
# Cell 11: Create vector search index on the Delta table
from databricks.vector_search.client import VectorSearchClient

vsc = VectorSearchClient(disable_notice=True)

# Full 3-part names
index_name = "databricks_energy_rag.energy_rag.chunks_vector_index"
source_table = "databricks_energy_rag.energy_rag.document_chunks"
endpoint_name = "energy_rag_endpoint"

# Create the index
try:
    index = vsc.create_delta_sync_index(
        endpoint_name=endpoint_name,
        index_name=index_name,
        source_table_name=source_table,
        pipeline_type="TRIGGERED",
        primary_key="chunk_id",
        embedding_dimension=1536,
        embedding_vector_column="embedding"
    )
    print(f"Index created: {index_name}")
    print("Index is building in background (takes 2-3 minutes)...")
except Exception as e:
    print(f"Error: {e}")

In [0]:
# Cell 12: Check index status
import time

index_name = "databricks_energy_rag.energy_rag.chunks_vector_index"

vsc = VectorSearchClient(disable_notice=True)

for i in range(10):
    try:
        index = vsc.get_index(endpoint_name="energy_rag_endpoint", index_name=index_name)
        status = index.describe()
        print(f"Status: {status.get('status', {}).get('state', 'BUILDING')}")
        
        if status.get('status', {}).get('state') == 'ONLINE':
            print("Index is ONLINE and ready!")
            break
    except Exception as e:
        print(f"Checking... {e}")
    
    time.sleep(20)

In [0]:
# Cell 12a: Final status check
vsc = VectorSearchClient(disable_notice=True)
index = vsc.get_index(endpoint_name="energy_rag_endpoint", 
                      index_name="databricks_energy_rag.energy_rag.chunks_vector_index")

status = index.describe()
print(f"Current status: {status.get('status', {}).get('state', 'UNKNOWN')}")
print(f"Full status info: {status.get('status', {})}")

In [0]:
# Cell 13: RAG query function
def query_rag_system(question, top_k=5):
    """
    Query the RAG system with a question
    Returns relevant context and AI-generated answer
    """
    
    # Step 1: Get embedding for the question
    question_embedding = get_embedding(question)
    
    # Step 2: Search vector index for similar chunks
    vsc = VectorSearchClient(disable_notice=True)
    index = vsc.get_index(
        endpoint_name="energy_rag_endpoint",
        index_name="databricks_energy_rag.energy_rag.chunks_vector_index"
    )
    
    results = index.similarity_search(
        query_vector=question_embedding,
        columns=["chunk_id", "document_name", "text"],
        num_results=top_k
    )
    
    # Step 3: Build context from retrieved chunks
    context_chunks = results.get('result', {}).get('data_array', [])
    context = "\n\n".join([chunk[2] for chunk in context_chunks])
    
    # Step 4: Generate answer using GPT with context
    response = client_eastus.chat.completions.create(
        model=GPT_DEPLOYMENT,
        messages=[
            {"role": "system", "content": "You are an expert on Alberta energy regulations. Answer questions based only on the provided context."},
            {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {question}\n\nAnswer:"}
        ],
        max_tokens=500
    )
    
    return {
        "question": question,
        "answer": response.choices[0].message.content,
        "sources": [chunk[1] for chunk in context_chunks],
        "context": context
    }

print("RAG query function ready")

In [0]:
# Cell 13: RAG query function
def get_embedding(text):
    response = client_canada.embeddings.create(
        model=EMBEDDING_DEPLOYMENT,
        input=text
    )
    return response.data[0].embedding

def query_rag_system(question, top_k=5):
    """
    Query the RAG system with a question
    Returns relevant context and AI-generated answer
    """
    
    # Step 1: Get embedding for the question
    question_embedding = get_embedding(question)
    
    # Step 2: Search vector index for similar chunks
    vsc = VectorSearchClient(disable_notice=True)
    index = vsc.get_index(
        endpoint_name="energy_rag_endpoint",
        index_name="databricks_energy_rag.energy_rag.chunks_vector_index"
    )
    
    results = index.similarity_search(
        query_vector=question_embedding,
        columns=["chunk_id", "document_name", "text"],
        num_results=top_k
    )
    
    # Step 3: Build context from retrieved chunks
    context_chunks = results.get('result', {}).get('data_array', [])
    context = "\n\n".join([chunk[2] for chunk in context_chunks])
    
    # Step 4: Generate answer using GPT with context
    response = client_eastus.chat.completions.create(
        model=GPT_DEPLOYMENT,
        messages=[
            {"role": "system", "content": "You are an expert on Alberta energy regulations. Answer questions based only on the provided context."},
            {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {question}\n\nAnswer:"}
        ],
        max_tokens=500
    )
    
    return {
        "question": question,
        "answer": response.choices[0].message.content,
        "sources": [chunk[1] for chunk in context_chunks],
        "context": context
    }

print("RAG query function ready")

In [0]:
# Cell 14: Test RAG system
test_question = "What are the requirements for flaring in Alberta oil and gas operations?"

print(f"Question: {test_question}\n")
print("Querying RAG system...\n")

result = query_rag_system(test_question, top_k=3)

print("=" * 80)
print(f"ANSWER:\n{result['answer']}")
print("=" * 80)
print(f"\nSOURCES: {', '.join(set(result['sources']))}")
print(f"\nRetrieved {len(result['sources'])} relevant chunks")

In [0]:
# Cell 15: Test multiple questions
questions = [
    "What are the measurement requirements for oil production?",
    "What regulations apply to upstream well drilling?",
    "What is Directive 017 about?"
]

for q in questions:
    print(f"\nQuestion: {q}")
    result = query_rag_system(q, top_k=3)
    print(f"Answer: {result['answer'][:200]}...")
    print(f"Sources: {set(result['sources'])}\n")
    print("-" * 80)

In [0]:
# Cell 16: Install Gradio
%pip install gradio
dbutils.library.restartPython()

In [0]:

# Cell 17: Simple interactive query interface
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Create widgets
question_input = widgets.Textarea(
    value='',
    placeholder='Ask a question about Alberta energy regulations...',
    description='Question:',
    layout=widgets.Layout(width='80%', height='80px')
)

submit_button = widgets.Button(
    description='Get Answer',
    button_style='primary',
    layout=widgets.Layout(width='200px')
)

output_area = widgets.Output()

def on_submit(b):
    with output_area:
        clear_output()
        question = question_input.value
        
        if not question.strip():
            print("Please enter a question.")
            return
        
        print(f"Question: {question}\n")
        print("Searching documents...")
        
        try:
            result = query_rag_system(question, top_k=3)
            
            print("\n" + "="*80)
            print("ANSWER:")
            print("="*80)
            print(result['answer'])
            print("\n" + "="*80)
            print(f"SOURCES: {', '.join(set(result['sources']))}")
            print("="*80)
            
        except Exception as e:
            print(f"Error: {str(e)}")

submit_button.on_click(on_submit)

# Display interface
display(HTML("<h2>Alberta Energy Regulations RAG Assistant</h2>"))
display(HTML("<p>Ask questions about AER Directive 017 (Measurement) and Directive 060 (Flaring)</p>"))
display(question_input)
display(submit_button)
display(output_area)

print("\nExample questions:")
print("- What are the flaring requirements?")
print("- What does Directive 017 cover?")
print("- What are measurement requirements for oil production?")

In [0]:
# Cell 18: Architecture Documentation
print("""
================================================================================
ALBERTA ENERGY RAG SYSTEM - ARCHITECTURE
================================================================================

COMPONENTS:
-----------
1. Data Ingestion
   - Source: Alberta Energy Regulator (AER) public PDFs
   - Documents: Directive 017 (Measurement), Directive 060 (Flaring)
   - Storage: /tmp on Databricks cluster

2. Document Processing
   - PDF text extraction: PyPDF2
   - Chunking: 1000 chars with 200 char overlap
   - Total chunks: 1,396

3. Embedding Generation
   - Model: Azure OpenAI text-embedding-3-small
   - Dimensions: 1536
   - Location: Canada East region
   - Cost: ~$2 for 1.4M tokens

4. Vector Storage & Search
   - Storage: Unity Catalog Delta table
   - Table: databricks_energy_rag.energy_rag.document_chunks
   - Vector Index: Databricks Vector Search (HNSW algorithm)
   - Endpoint: energy_rag_endpoint
   - Index sync: Delta Change Data Feed enabled

5. Query & Generation
   - Query embedding: Same as step 3
   - Similarity search: Top-K retrieval (default K=3)
   - LLM: Azure OpenAI gpt-4o-mini
   - Location: East US 2 region
   - Context window: Retrieved chunks + question

DATA FLOW:
----------
User Question 
  → Embed question (Azure OpenAI Canada East)
  → Search vector index (Databricks Vector Search)
  → Retrieve top-K similar chunks
  → Build context from chunks
  → Generate answer (Azure OpenAI East US 2 + context)
  → Return answer + sources

KEY FEATURES:
-------------
- Unified lakehouse: No separate vector database needed
- Governance: Unity Catalog handles permissions & lineage
- Cost efficient: Delta storage + triggered index updates
- Scalable: Can handle millions of documents
- Explainable: Always cites source documents

DEPLOYMENT:
-----------
- Cluster: Single-node Standard_DS3_v2 (auto-terminate 30min)
- Runtime: Databricks 15.4 LTS
- Compute cost: ~$0.55/DBU hour + VM cost


================================================================================
""")

In [0]:
# Cell 19: Generate README content for GitHub
readme_content = """
# Alberta Energy Regulations RAG Assistant

AI-powered question-answering system for Alberta Energy Regulator directives using Retrieval-Augmented Generation (RAG).

## Business Problem
Energy companies and regulatory professionals spend hours searching through dense regulatory documents. This RAG system provides instant, accurate answers grounded in official AER directives.

## Tech Stack
- **Azure Databricks**: Unity Catalog, Delta Lake, Vector Search
- **Azure OpenAI**: text-embedding-3-small, gpt-4o-mini
- **Data Source**: Alberta Energy Regulator public directives

## Architecture
```
User Question → Embedding → Vector Search → Context Retrieval → LLM Generation → Answer
```

## Key Features
1. **Unified Lakehouse**: Single Delta table stores text + embeddings + metadata
2. **Databricks Vector Search**: No separate vector database needed
3. **Source Attribution**: Every answer cites specific documents
4. **Cost Efficient**: ~$15 total development cost, scales to production

## Documents Indexed
- AER Directive 017: Measurement Requirements (866K chars)
- AER Directive 060: Upstream Petroleum Industry Flaring (250K chars)
- Total: 1,396 chunks with 1536-dimensional embeddings

## Demo Queries
- "What are the flaring requirements in Alberta?"
- "What does Directive 017 cover?"
- "What are measurement requirements for oil production?"

## Technical Highlights
- **Delta Lake**: ACID transactions, time travel, Change Data Feed
- **Unity Catalog**: Centralized governance and lineage tracking
- **Vector Search**: HNSW index for sub-second similarity search
- **Multi-region**: Optimized endpoints (Canada East + East US 2)


## Metrics
- Query latency: 2-3 seconds end-to-end
- Accuracy: Answers grounded in source text
- Scalability: Tested with 1.4K chunks, scales to millions

## Future Enhancements
- Add more AER directives and regulations
- Multi-modal: Include diagrams and tables from PDFs
- Real-time updates: Auto-sync when new directives published
- Fine-tuned embeddings for domain-specific terminology


"""

# Save to file for easy copy-paste
with open("/tmp/README.md", "w") as f:
    f.write(readme_content)

print("README.md content created at /tmp/README.md")
print("\n" + "="*80)
print(readme_content)